In [1]:
%reload_ext autotime
import geopandas as gpd
import pandas as pd
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
from coastsat import SDS_transects
pd.options.plotting.backend = "plotly"

In [2]:
# Transects, origin is landward. Has beach_slope
transects = gpd.read_file("transects_extended.geojson")
transects.set_index("id", inplace=True)
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.441081,767.0,428.0,0.168420,28.102591,1263.560863,35.546601,179.085729,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-1.037105,767.0,569.0,0.097874,25.419324,1033.770813,32.152306,212.247788,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.680019,767.0,588.0,0.053927,22.632907,838.007507,28.948359,205.106151,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,-0.405198,767.0,598.0,0.023412,20.749758,698.653187,26.432048,191.745881,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,-0.090025,767.0,608.0,0.001277,19.889328,655.810616,25.608800,175.092121,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0580-0005,nzd0580,NaN,NaN,NaN,0.130,0.1009,0.1853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (172.69317 -43.62862, 172.69259 -43..."
nzd0580-0006,nzd0580,NaN,NaN,NaN,0.045,0.0387,0.0536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (172.69079 -43.6289, 172.69309 -43...."
nzd0580-0007,nzd0580,NaN,NaN,NaN,0.025,0.0240,0.0254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,"LINESTRING (172.68991 -43.62923, 172.69221 -43..."


In [3]:
vos_files = pd.Series(
    sorted(glob("csv_run7/*/time_series_tidally_corrected.csv"))
)
vos_files = vos_files[~vos_files.str.contains("nzd")]
vos_files

0       csv_run7/aus0001/time_series_tidally_corrected...
1       csv_run7/aus0002/time_series_tidally_corrected...
2       csv_run7/aus0003/time_series_tidally_corrected...
3       csv_run7/aus0004/time_series_tidally_corrected...
4       csv_run7/aus0005/time_series_tidally_corrected...
                              ...                        
3066    csv_run7/usa_HI_0053/time_series_tidally_corre...
3067    csv_run7/usa_HI_0054/time_series_tidally_corre...
3068    csv_run7/usa_HI_0055/time_series_tidally_corre...
3069    csv_run7/usa_HI_0056/time_series_tidally_corre...
3070    csv_run7/usa_HI_0057/time_series_tidally_corre...
Length: 2511, dtype: object

In [4]:
my_files = pd.Series(
    sorted(glob("data/*/transect_time_series_tidally_corrected.csv"))
)
my_files

0      data/ber0001/transect_time_series_tidally_corr...
1      data/ber0002/transect_time_series_tidally_corr...
2      data/nzd0001/transect_time_series_tidally_corr...
3      data/nzd0002/transect_time_series_tidally_corr...
4      data/nzd0003/transect_time_series_tidally_corr...
                             ...                        
576    data/nzd0576/transect_time_series_tidally_corr...
577    data/nzd0577/transect_time_series_tidally_corr...
578    data/nzd0578/transect_time_series_tidally_corr...
579    data/nzd0579/transect_time_series_tidally_corr...
580    data/nzd0580/transect_time_series_tidally_corr...
Length: 581, dtype: object

In [5]:
sar_files = pd.Series(sorted(glob("data/sar*/transect_time_series.csv")))
sar_files

0       data/sar0001/transect_time_series.csv
1       data/sar0002/transect_time_series.csv
2       data/sar0003/transect_time_series.csv
3       data/sar0004/transect_time_series.csv
4       data/sar0005/transect_time_series.csv
                        ...                  
2536    data/sar2537/transect_time_series.csv
2537    data/sar2538/transect_time_series.csv
2538    data/sar2539/transect_time_series.csv
2539    data/sar2540/transect_time_series.csv
2540    data/sar2541/transect_time_series.csv
Length: 2541, dtype: object

In [6]:
files = pd.concat([vos_files, my_files, sar_files])
files

0       csv_run7/aus0001/time_series_tidally_corrected...
1       csv_run7/aus0002/time_series_tidally_corrected...
2       csv_run7/aus0003/time_series_tidally_corrected...
3       csv_run7/aus0004/time_series_tidally_corrected...
4       csv_run7/aus0005/time_series_tidally_corrected...
                              ...                        
2536                data/sar2537/transect_time_series.csv
2537                data/sar2538/transect_time_series.csv
2538                data/sar2539/transect_time_series.csv
2539                data/sar2540/transect_time_series.csv
2540                data/sar2541/transect_time_series.csv
Length: 5633, dtype: object

In [7]:
f = files[files.str.contains("ber0001")].iloc[0]
# despiked_filename = f.replace(".csv", "_tidally_corrected.csv")
df = pd.read_csv(f)
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
display(df.columns)
import matplotlib.pyplot as plt

transect_id = "ber0001-0002"


def custom_mean(window):
    return window[window.between(window.quantile(0.25), window.quantile(0.75))].mean()


pd.DataFrame(
    {
        "raw": df[transect_id],
        "rolling 90d mean": df[transect_id].rolling("90d", min_periods=1).mean(),
        "rolling 180d mean": df[transect_id].rolling("180d", min_periods=1).mean(),
        "rolling 90d custom mean": df[transect_id]
        .rolling("90d", min_periods=1)
        .apply(custom_mean),
        "rolling 180d custom mean": df[transect_id]
        .rolling("180d", min_periods=1)
        .apply(custom_mean),
        # "rolling 365d": df[transect_id].rolling("365d", min_periods=1).mean(),
    },
    index=df.index,
).plot()

Index(['ber0001-0000', 'ber0001-0001', 'ber0001-0002', 'ber0001-0003',
       'ber0001-0004', 'ber0001-0005', 'ber0001-0006', 'ber0001-0007',
       'ber0001-0008', 'ber0001-0009', 'satname'],
      dtype='object')

In [8]:
df = pd.read_csv("data/sar0939/transect_time_series.csv")
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
(df["sar0939-0000"] - 93).plot()

In [9]:
def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)


def get_trends(f):
    df = pd.read_csv(f)
    try:
        df.dates = pd.to_datetime(df.dates)
    except:
        print(f)
    if "sar" in f or "ber" in f:
        smoothed_filename = f.replace(".csv", "_smoothed.csv")
        try:
            df = pd.read_csv(smoothed_filename)
            df.dates = pd.to_datetime(df.dates)
        except:
            df.dates = pd.to_datetime(df.dates)
            df.set_index("dates", inplace=True)
            satname = df.satname
            df = df.drop(columns="satname").apply(despike, axis=0)
            df["satname"] = satname
            df.reset_index(names="dates").to_csv(
                f.replace(".csv", "_despiked.csv"), index=False
            )
            for transect_id in df.drop(columns="satname").columns:
                df[transect_id] = df[transect_id].rolling("180d", min_periods=1).mean()
            df.reset_index(names="dates", inplace=True)
            df.to_csv(f.replace(".csv", "_smoothed.csv"), index=False)
    df.index = (df.dates - df.dates.min()).dt.days / 365.25
    df.drop(columns=["dates", "satname", "Unnamed: 0"], inplace=True, errors="ignore")
    trends = []
    for transect_id in df.columns:
        sub_df = df[transect_id].dropna()
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1, 1)
        y = sub_df
        linear_model = LinearRegression().fit(x, y)
        pred = linear_model.predict(x)
        trends.append(
            {
                "transect_id": transect_id,
                "trend": linear_model.coef_[0],
                "intercept": linear_model.intercept_,
                "n_points": len(df[transect_id]),
                "n_points_nonan": len(sub_df),
                "r2_score": r2_score(y, pred),
                "mae": mean_absolute_error(y, pred),
                "mse": mean_squared_error(y, pred),
                "rmse": root_mean_squared_error(y, pred),
            }
        )
    return pd.DataFrame(trends)


# trends = get_trends(sar_files.iloc[-1]).set_index("transect_id")
trends = pd.concat(process_map(get_trends, my_files)).set_index("transect_id")
len(trends)

  0%|          | 0/581 [00:00<?, ?it/s]

32678

In [10]:
trends[trends.n_points_nonan > 10].sort_values("r2_score")

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
transect_id,,,,,,,,
nzd0557-0004,0.000071,284.085638,240,182,1.192189e-09,12.048177,242.119347,15.560185
nzd0554-0011,-0.000078,305.445973,435,326,1.197034e-09,13.068167,293.598747,17.134723
nzd0282-0125,-0.000100,345.832300,743,501,1.521247e-09,15.877540,399.513017,19.987822
nzd0537-0024,0.000097,337.690080,403,288,1.654758e-09,14.000048,327.086462,18.085532
nzd0217-0304,-0.000109,378.009695,591,217,5.552209e-09,9.383411,135.300572,11.631877
...,...,...,...,...,...,...,...,...
nzd0121-0007,13.893157,181.582307,173,113,9.426559e-01,20.324738,665.833940,25.803758
nzd0121-0002,15.323605,247.981605,173,111,9.431992e-01,22.029951,789.021516,28.089527
nzd0121-0003,14.527905,243.591174,173,116,9.444384e-01,19.819405,676.385860,26.007419


In [11]:
trends[trends.index.str.startswith("ber")]

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
transect_id,,,,,,,,
ber0001-0000,-0.462662,172.942131,217,217,4.091450e-01,3.194084,19.180097,4.379509
ber0001-0001,-0.377595,164.045559,217,217,4.158845e-01,2.616221,12.425027,3.524915
ber0001-0002,-0.237611,162.250470,217,217,1.909726e-01,2.813023,14.840370,3.852320
ber0001-0003,-0.231374,162.928497,217,217,1.937340e-01,2.681119,13.823584,3.718008
ber0001-0004,-0.244762,163.289489,217,217,1.915469e-01,3.092245,15.688707,3.960897
ber0001-0005,-0.203432,160.599356,217,217,1.253129e-01,3.317711,17.923198,4.233580
ber0001-0006,-0.104889,157.413329,217,217,3.142298e-02,3.673513,21.040842,4.587030
ber0001-0007,-0.156261,160.676732,217,217,9.869806e-02,2.977771,13.835097,3.719556
ber0001-0008,-0.392455,166.533154,217,217,5.216501e-01,2.195485,8.763265,2.960281


In [12]:
trends.describe()

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,32678.000000,32678.000000,32678.000000,32678.000000,3.267800e+04,32678.000000,32678.000000,32678.000000
mean,0.114964,350.587629,448.112920,333.066559,1.503593e-01,12.364238,355.859327,15.789276
std,1.850969,55.724554,160.712967,125.117041,1.933056e-01,8.350714,829.309127,10.322856
min,-39.203896,-168.169906,103.000000,10.000000,1.192189e-09,2.195485,8.763265,2.960281
25%,-0.389864,327.144212,341.000000,251.000000,1.426662e-02,8.060385,107.599483,10.373017
50%,0.071719,348.585594,430.000000,325.000000,6.599016e-02,10.116513,166.591411,12.907030
75%,0.560581,371.752395,568.000000,423.000000,2.089449e-01,13.549166,296.794148,17.227715
max,26.713175,1153.153704,1038.000000,885.000000,9.513573e-01,134.625990,37328.144351,193.204928


In [13]:
(transects.trend - trends.trend).describe()

count    32668.000000
mean        -0.000010
std          0.014320
min         -1.038414
25%          0.000000
50%          0.000000
75%          0.000000
max          0.492585
Name: trend, dtype: float64

In [14]:
transects.update(trends.drop_duplicates())

In [15]:
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.441081,767.0,428.0,0.168420,28.102591,1263.560863,35.546601,179.085729,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-1.037105,767.0,569.0,0.097874,25.419324,1033.770813,32.152306,212.247788,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.680019,767.0,588.0,0.053927,22.632907,838.007507,28.948359,205.106151,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,-0.405198,767.0,598.0,0.023412,20.749758,698.653187,26.432048,191.745881,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,-0.090025,767.0,608.0,0.001277,19.889328,655.810616,25.608800,175.092121,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0580-0005,nzd0580,NaN,NaN,NaN,0.130,0.1009,0.1853,-0.018333,685.0,548.0,0.000104,10.414801,192.405646,13.871036,217.693704,None,"LINESTRING (172.69317 -43.62862, 172.69259 -43..."
nzd0580-0006,nzd0580,NaN,NaN,NaN,0.045,0.0387,0.0536,0.095100,685.0,514.0,0.000880,17.674378,612.825349,24.755310,177.943022,None,"LINESTRING (172.69079 -43.6289, 172.69309 -43...."
nzd0580-0007,nzd0580,NaN,NaN,NaN,0.025,0.0240,0.0254,-0.158103,685.0,524.0,0.004666,13.761916,315.201091,17.753904,207.218599,None,"LINESTRING (172.68991 -43.62923, 172.69221 -43..."


In [16]:
trends.columns, transects.columns, trends.columns.isin(transects.columns)

(Index(['trend', 'intercept', 'n_points', 'n_points_nonan', 'r2_score', 'mae',
        'mse', 'rmse'],
       dtype='object'),
 Index(['site_id', 'orientation', 'along_dist', 'along_dist_norm',
        'beach_slope', 'cil', 'ciu', 'trend', 'n_points', 'n_points_nonan',
        'r2_score', 'mae', 'mse', 'rmse', 'intercept', 'ERODIBILITY',
        'geometry'],
       dtype='object'),
 array([ True,  True,  True,  True,  True,  True,  True,  True]))

In [17]:
transects = transects.join(trends.loc[:, ~trends.columns.isin(transects.columns)])
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.441081,767.0,428.0,0.168420,28.102591,1263.560863,35.546601,179.085729,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-1.037105,767.0,569.0,0.097874,25.419324,1033.770813,32.152306,212.247788,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.680019,767.0,588.0,0.053927,22.632907,838.007507,28.948359,205.106151,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,-0.405198,767.0,598.0,0.023412,20.749758,698.653187,26.432048,191.745881,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,-0.090025,767.0,608.0,0.001277,19.889328,655.810616,25.608800,175.092121,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0580-0005,nzd0580,NaN,NaN,NaN,0.130,0.1009,0.1853,-0.018333,685.0,548.0,0.000104,10.414801,192.405646,13.871036,217.693704,None,"LINESTRING (172.69317 -43.62862, 172.69259 -43..."
nzd0580-0006,nzd0580,NaN,NaN,NaN,0.045,0.0387,0.0536,0.095100,685.0,514.0,0.000880,17.674378,612.825349,24.755310,177.943022,None,"LINESTRING (172.69079 -43.6289, 172.69309 -43...."
nzd0580-0007,nzd0580,NaN,NaN,NaN,0.025,0.0240,0.0254,-0.158103,685.0,524.0,0.004666,13.761916,315.201091,17.753904,207.218599,None,"LINESTRING (172.68991 -43.62923, 172.69221 -43..."


In [18]:
transects[transects.site_id.str.startswith("sar") & ~transects.trend.isna()]

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
sar0001-0000,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.209107,671.0,667.0,0.003085,34.337637,2024.899412,44.998882,135.992639,Medium,"LINESTRING (8.40852 38.86175, 8.40882 38.86535)"
sar0001-0001,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.168722,671.0,667.0,0.008700,11.931958,460.730301,21.464629,197.877604,Medium,"LINESTRING (8.4084 38.86162, 8.41092 38.86464)"
sar0001-0002,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.012318,671.0,669.0,0.000413,5.801157,51.994338,7.210710,206.410952,Medium,"LINESTRING (8.40893 38.86153, 8.41236 38.86393)"
sar0001-0003,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.043548,671.0,669.0,0.016188,3.202754,16.331152,4.041182,239.500879,Medium,"LINESTRING (8.40904 38.86129, 8.41333 38.8626)"
sar0001-0004,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.011825,671.0,669.0,0.001501,2.692829,13.178444,3.630213,258.136097,Medium,"LINESTRING (8.40904 38.86176, 8.4133 38.86037)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2541-0000,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,-0.297976,1371.0,1371.0,0.203601,5.662483,50.986351,7.140473,199.127124,High,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
sar2541-0001,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.011742,1371.0,1371.0,0.000671,3.998349,30.150238,5.490923,165.933848,High,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
sar2541-0002,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.048766,1371.0,1371.0,0.008370,4.829729,41.363000,6.431407,161.418975,High,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"


In [19]:
transects.reset_index().drop_duplicates(subset="id").to_file("transects_extended.geojson", driver="GeoJSON")